# Open Enrollment
Load and join open enrollment files for 2005-06 through the current year.

In [4]:
import pandas as pd
import re
import datetime

#Source folder for the excel files, downloaded from MDE website
folder = 'C:/Users/swastvedt/Documents/MDE Data_Documents/OpenEnrollment/Originals/'
currentYear = datetime.date.today().year % 100
#Earliest year we have data for
a = 9

openEnroll = pd.DataFrame()
files = []

###Create a list of file names
#Account for change in file name format that occurred in 2015-16
while a < currentYear:
    if a < 10:
        files.append(folder + 'open enrollment 0' + str(a) + '-' + str(a+1) + '.xlsx')
        
    else:
        files.append(folder + 'open enrollment ' + str(a) + '-' + str(a+1) + '.xlsx')
        
    a = a + 1
    
def upcase_first_letter(s):
    return s[0].upper() + s[1:]

def remove_spaces(s):
    return re.sub(r' ', '', s)

###Capitalize column names and create "DistrictID" column to use as an index
def clean_table(df):
    df.columns = [str(upcase_first_letter(x)) for x in df.columns]
    df.columns = [str(remove_spaces(x)) for x in df.columns]
    df.rename(columns = {'Dat_yer':'DataYear'}, inplace=True)
    df['EnrollDistrictID'] = df['EnrollingDistrictNumber'] + '-' + df['EnrollingDistrictType'] + '-000'
    df['ResidentDistrictID'] = df['ResidentDistrictNumber'] + '-' + df['ResidentDistrictType'] + '-000'
    df = df.set_index(['DataYear', 'EnrollDistrictID', 'ResidentDistrictID'])
    
    return df

###Load files from source folder. Create a list of dataframes for each year that includes state, district and school-level information
def load_data(f):
    df = pd.read_excel(f, dtype = {0:str, 1: str, 2: str, 5: str, 6: str})
    df = clean_table(df)
    
    return df

In [5]:
###Load data for each file name in the files list
frames = [ load_data(f) for f in files ]

In [6]:
###Join dataframes for all years into one. Drop blank rows.
openEnroll = openEnroll.append(frames)

In [7]:
openEnroll.drop('end of workbook', level='DataYear', inplace=True)

In [8]:
###Save raceEthnicity table to local folder
openEnroll.to_csv('tempdata/openEnroll.csv')

In [9]:
###Update shared file on OneDrive and return new link to file.
import onedrivesdk
from onedrivesdk.helpers import GetAuthCodeServer

redirect_uri = 'http://localhost:8080/'

#Credentials for "Python data upload" OneDrive app
client_secret = ***
client = onedrivesdk.get_default_client(client_id=***,
                                            scopes=['wl.signin',
                                                    'wl.offline_access',
                                                    'onedrive.readwrite'])
auth_url = client.auth_provider.get_auth_url(redirect_uri)

# Block thread until we have the code
code = GetAuthCodeServer.get_auth_code(auth_url, redirect_uri)
# Finally, authenticate
client.auth_provider.authenticate(code, redirect_uri, client_secret)

127.0.0.1 - - [28/Sep/2017 15:30:06] "GET /?code=M6c1d1aed-2a43-5d24-bd03-3e43c7a15839 HTTP/1.1" 200 -


In [10]:
returned_item = client.item(id='root').children['openEnroll.csv'].upload('tempdata/openEnroll.csv')

In [11]:
#Input the client information from above and the "openEnroll.csv" filename to generate a new sharing link
def get_sharing_link(c, fileName):
    permission = c.item(id='root').children[fileName].create_link("view").post()
    return ("\n{}\n".format(permission.link.web_url))

get_sharing_link(client, 'openEnroll.csv')

'\nhttps://1drv.ms/u/s!AvY_lznp4oOFbYO05RJAN2MP-90\n'